## 웹 크롤링

### 인터넷 접속

- urllib.request 모듈로 작업
- HTML 소스 로드로 별다른 문제 없음


In [58]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색 함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]

    # 기온 가져오기
    for i in range(7):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

    # 습도 가져오기
    for i in range(3):
        text = text[text.find('<td>')+1:]

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')
    
get_weather('부산')

부산의 현재 기온은 12.2˚C 입니다.
부산의 현재 습도는 24% 입니다.


### OpneAPI 크롤링

- 공공데이터 포털
    - http://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

```python
response = requests.get(total_url, verify=False)
```

- 문제점
    - https를 일반 request로 부르면 SSL 오류 발생
    - 해결하려면 외무모듈 requests를 사용, verify = False 옵션으로 지정 (verify는 검증을 하냐마냐 여부)
    - 아래 셀의 19번 라인!

In [53]:
import requests
from urllib.parse import quote, unquote,urlencode  # 한글을 URL인코드로 변환해주는 함수
import json
import ssl
# stationName 정류소 이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    queryString = "?" + urlencode(
        {    
            'serviceKey': 'EmCpfwaIYY1h6xUdXPg6c9DxsctH9Bau9ueNoGXoAs0zEckxoWbkI1sdOY/4+p6z3so0n51cffoU0q5ha+hQMQ==',
            'pageNo': '1',
            'numOfRows': '10',
            'resultType': type,
            'stationLoc' : stationName
        }
    )

    total_url = api_url + queryString
    response = requests.get(total_url, verify=False)
    return response.text
        
try:
    result = getDataPortalSearch('부경대','json')
    json_data = json.loads(result)
    station_data = json_data['getTblBusinfoeqStusInfo']['body']['items']['item']

    for item in station_data:
        print(item)
except Exception as e:
    print('해당 데이터가 없습니다.')



{'stationNum': '07006', 'stationLoc': '경성대.부경대역', 'lat': '35.13856124', 'lng': '129.1023074', 'addr': '남구 대연3동 93-7', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07007', 'stationLoc': '경성대.부경대역', 'lat': '35.13837971', 'lng': '129.1024864', 'addr': '남구 대연3동 90-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07011', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13287689', 'lng': '129.1012804', 'addr': '남구 대연3동 531-2', 'insYear': '', 'dataDay': '2023-02-03'}
{'stationNum': '07012', 'stationLoc': '부경대대연캠퍼스', 'lat': '35.13234515', 'lng': '129.1010586', 'addr': '남구 대연3동 537-1', 'insYear': '', 'dataDay': '2023-02-03'}


### OpenAPI로 가져온 데이터 지도 표시

```python
stop_str = '<h4>'+item['stationLoc']+'</h4>' + item['stationNum']+ '<br>' + item['addr']
```
- 문제점
    - 딕셔너리 구조하고 문자열 포맷팅 f'{}' 호환 안됨! (바로 위에 적어둔 파이썬 코드)
    - 구식 방법인 문자열 결합 방식으로 해결

In [54]:
import folium

# 정류소 중 제일 첫번째 인덱스 정류소 위경도를 중심으로 잡음
if len(station_data) > 0: 
    center_lat = station_data[0]['lat']
    center_lng = station_data[0]['lng']

m = folium.Map(location = [center_lat, center_lng], zoom_start = 13)

# 전체 정류소 위치값을 마커로 표시
for item in station_data:

    stop_str = '<h4>'+item['stationLoc']+'</h4>' + item['stationNum']+ '<br>' + item['addr']
    iframe = folium.IFrame(stop_str)
    popup = folium.Popup(iframe, min_width = 200, max_width = 200)
    
    folium.Marker(location =[item['lat'], item['lng']], popup = popup, icon = folium.Icon(icon = 'pushpin')).add_to(m)

m

## BeautifulSoup (Version 4)
웹크롤링을 편하게 해주는 도구(라이브러리)

```python
pip install beautifulsoup4
```

In [59]:
!pip install beautifulsoup4

     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     ------------------------------------ - 122.9/129.4 kB 7.5 MB/s eta 0:00:01
     -------------------------------------- 129.4/129.4 kB 3.8 MB/s eta 0:00:00


#### 모듈 import

```python
from bs4 import BeautifulSoup
```

In [60]:
from bs4 import BeautifulSoup
import requests

url = 'https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4+%EC%9B%B9+%EB%A7%9D%EC%9B%90%EA%B2%BD'

response = requests.get(url)
response.text

'\n\n\n\n\n\n\n\n\n\n\n\n\t\n\t\n\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="ko">\n<head>\n<meta name="referrer" contents="always">\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="description" content="제임스 웹 망원경의 지식iN Q&amp;A 검색결과입니다. 궁금증을 해결하지 못했다면 지식iN &#039;질문하기&#039;를 해보세요."/>\n<meta name="viewport" content="width=1024">\n\n<meta name="msapplication-config" content="none" />\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/common.css" />\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/components.css" />\n\n\n\t\n\t\n\t\t<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/static.kin/static/pc/20230201140250/css/min/other.css">\n\t\n\n\n\n\n\n\n<script>\n\t\n\tvar pcDomain = "kin.naver.com";\n\tvar pcDomainWit